In [1]:
import pandas as pd
import numpy as np

In [2]:
filepath = "malware_detection_dataset.csv"

In [3]:
df = pd.read_csv(filepath)

### Split the Dataset into X and y

In [4]:
X = df.drop('label',axis=1)
y = df['label']

In [5]:
X

,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,proto_icmp,proto_tcp,...,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,0.114184,48.0,48.0,0,1,76,1,76,0,0,...,0,0,0,0,0,0,0,1,0,0
1,160.367579,7536.0,0.0,0,24,8208,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0.016986,48.0,48.0,0,1,76,1,76,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0.003497,0.0,0.0,0,5,212,3,144,0,1,...,0,0,0,0,0,0,0,1,0,0
4,0.036724,34.0,311.0,0,1,62,1,339,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046906,0.012744,96.0,96.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1046907,0.012744,96.0,96.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1046908,0.012744,96.0,96.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1046909,0.012744,96.0,96.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [6]:
y

0          benign
1          benign
2          benign
3          benign
4          benign
            ...  
1046906      DDoS
1046907      DDoS
1046908      DDoS
1046909      DDoS
1046910      DDoS
Name: label, Length: 1046911, dtype: object

### Splitting the X and y Dataset into the Training set and Test set

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 1)

In [8]:
y_test

222402                         Okiru
622557                         Okiru
435746                          DDoS
81474      PartOfAHorizontalPortScan
49984      PartOfAHorizontalPortScan
                     ...            
348333     PartOfAHorizontalPortScan
980993                          DDoS
779507     PartOfAHorizontalPortScan
620243                         Okiru
1027748                         DDoS
Name: label, Length: 209383, dtype: object

In [9]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(837528, 24)
(209383, 24)
(837528,)
(209383,)


### Feature Scaling

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [11]:
from sklearn.ensemble import AdaBoostClassifier

import timeit
start = timeit.default_timer()


AB_clf = AdaBoostClassifier()  # Create Random Forest classifier object
AB_clf.fit(X_train,y_train)  # Train Random Forest Classifier
AB_pred  = AB_clf.predict(X_test)  #Predict the response for test dataset
print(AB_pred)
print()

end = timeit.default_timer()
print("total runtime of the program is: ",end-start,'seconds')

['Okiru' 'Okiru' 'PartOfAHorizontalPortScan' ...
 'PartOfAHorizontalPortScan' 'Okiru' 'PartOfAHorizontalPortScan']

total runtime of the program is:  289.4068744 seconds


In [12]:
##Confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, AB_pred)
print(cm)


[[    0     0     0     0     0     0     0     0     0     0    26   777]
 [    0     0     0     0     0     0     0     0     0     0  2970    10]
 [    0     0     0     0     0     0     0     0     0     0     0     2]
 [    0     0     0     0     0     0     0     0     0     0    72     0]
 [    0     0     0     0     0     0     0     0     0     0     0     1]
 [    0     0     0     0     0     0     0     0     0     0     1     0]
 [    0     0     0     0     0     0     0     0     0     0     5     0]
 [    0     0     0     0     0     0     0     6     0    20 22630     0]
 [    0     0     0     0     0     0     0     0     0     0     0     1]
 [    0     0     0     0     0     0     0     0     0 39796     1    60]
 [    0     0     0     0     0     0     0     3     0 19779 89519   272]
 [    0     0     0     0     0     0     0     5     0  7416 23617  2394]]


In [13]:
### Final Results
from sklearn.metrics import classification_report
print(classification_report(y_test,AB_pred,zero_division=0))

                            precision    recall  f1-score   support

                    Attack       0.00      0.00      0.00       803
                       C&C       0.00      0.00      0.00      2980
          C&C-FileDownload       0.00      0.00      0.00         2
             C&C-HeartBeat       0.00      0.00      0.00        72
C&C-HeartBeat-FileDownload       0.00      0.00      0.00         1
                 C&C-Mirai       0.00      0.00      0.00         1
                 C&C-Torii       0.00      0.00      0.00         5
                      DDoS       0.43      0.00      0.00     22656
              FileDownload       0.00      0.00      0.00         1
                     Okiru       0.59      1.00      0.74     39857
 PartOfAHorizontalPortScan       0.64      0.82      0.72    109573
                    benign       0.68      0.07      0.13     33432

                  accuracy                           0.63    209383
                 macro avg       0.20      0.1

In [14]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, AB_pred)

0.629062531342086